In [14]:
!git pull

Already up to date.


In [15]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	fra.txt

nothing added to commit but untracked files present (use "git add" to track)


In [16]:
'''
  Setup tensorflow and other imports
'''
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

import pandas as pd

Found GPU at: /device:GPU:0


In [34]:
'''
  Data processing
'''
lines = pd.read_table('fra.txt', names=['eng', 'fra', 'other'])
lines = lines.drop('other', axis=1)

# Convert all to lowercase
lines = lines.applymap(lambda x: x.lower())

# Remove special characters

print(lines.head(50))

       eng                             fra
0      go.                            va !
1      go.                         marche.
2      go.                         bouge !
3      hi.                         salut !
4      hi.                          salut.
5     run!                         cours !
6     run!                        courez !
7     run!  prenez vos jambes à vos cous !
8     run!                          file !
9     run!                         filez !
10    run!                         cours !
11    run!                         fuyez !
12    run!                        fuyons !
13    run.                         cours !
14    run.                        courez !
15    run.  prenez vos jambes à vos cous !
16    run.                          file !
17    run.                         filez !
18    run.                         cours !
19    run.                         fuyez !
20    run.                        fuyons !
21    who?                           qui ?
22    wow! 